##### 적절한 인증 없는 중요 기능 허용 (p.68)

In [ ]:
from django.shortcuts import render
from re import escape
import hashlib

# login_required decorator를 사용하여 login된 사용자만 접근하도록 처리
@login_required
def change_password(request):
    new_pwd = request.POST.get('new_password','')
    crnt_pwd = request.POST.get('current_password','')

    # 세션에서 로그인한 사용자정보를 가져온다.
    user = '%s' % escape(request.session['userid'])

    crnt_h = hashlib.sha256(crnt_pwd.encode())
    h_pwd = crnt_h.hexdigest()

    # DB에서 기존 사용자의 Hash된 패스워드 가져오기
    old_pwd = get_password_from_db(user)

    # 패스워드를 변경하기 전 사용자에 대해 재인증한다.
    if old_pwd == h_pwd:
        new_h = hashlib.sha256(new_pwd.encode())
        update_password_from_db(user, new_h.hexdigest())
        return render(request, '/success.html')

##### 부적절한 인가 (p.70)

In [ ]:
from django.shortcuts import render
from .model import Content

@login_required
# 해당 기능을 수행할 권한이 있는지 확인
@permission_required('content.delete', raise_exception=True)
def delete_content(request):
    action = request.POST.get('action', '')
    content_id = request.POST.get('content_id', '')
    
    if action is not None and action == "delete":
        Content.objects.filter(id=content_id).delete()
        return render(request, '/success.html')
    else:
        return render(request, '/error.html', {'error':'삭제 실패'})

##### 중요한 자원에 대한 잘못된 권한 설정 (p.72)

In [ ]:
def write_file():
    #모든 사용자가 읽기, 쓰기, 실행 권한을 가지게 됨.
    os.chmod('/root/system_config', 0o700)
    
    with open("/root/system_config", 'w') as f:
        f.write("your config is broken")

##### 취약한 암호화 알고리즘 사용 (p.76)

1) DES 알고리즘 

In [ ]:
import base64
from Crypto.Cipher import AES

def get_enc_text(plain_text, key):
    # 안전한 알고리즘인 AES 를 사용하여 안전함.
    cipher_aes = AES.new(key, AES.MODE_CBC, iv)
    encrypted_data = base64.b64encode(cipher_des.encrypt(plain_text))
    return encrypted_data.decode('ASCII')

1) MD5 알고리즘 

In [ ]:
import hashlib

def make_md5(plain_text):
    # 안전한 sha-256 해쉬함수 사용
    hash_text = hashlib.sha256(plain_text.encode('utf-8')).hexdigest()
    return hash_text

##### 암호화되지 않은 중요정보 (p.72)

1) 중요정보 평문저장

In [ ]:
from Crypto.Hash import SHA256

def update_pass(dbconn, password, user_id, salt):
    # 단방향 암호화를 이용하여 비밀번호를 암호화
    hash_obj = SHA256.new()
    hash_obj.update(bytes(password + salt, 'utf-8'))
    hash_pwd = hash_obj.hexdigest()
    curs = dbconn.curs()
    curs.execute('UPDATE USERS SET PASSWORD=%s WHERE USER_ID=%s', (hash_pwd,
    user_id))

2) 중요정보 평문전송

In [ ]:
import socket
import os
from Crypto.Cipher import AES

HOST = '127.0.0.1'
PORT = 65434

def send_password(password):
    ......
    block_key = os.environ.get('BLOCK_KEY')
    
    aes = AEScipher(block_key)
    # 패스워드등 중요정보는 암호화하여 전송하는게 안전하다.
    enc_passowrd = aes.encrypt(passowrd)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(enc_passowrd.encode('utf-8'))
        data = s.recv(1024)
        .......

class AEScipher:
    BS = AES.block_size
    
    def __init__(self, s_key):
        self.s_key = s_key
    
    def pad(m):
        return m + bytes([BS - len(m) % BS] * (BS - len(m) % BS))
    
    def encrypt(self, plain):
        plain = pad(plain.encode())
        iv = os.environ.get('IV_KEY')
        cipher = AES.new(self.s_key, AES.MODE_CBC, iv)
        return (iv + cipher.encrypt(plain))
        ......

##### 하드코드된 중요정보 (p.81)

In [ ]:
import pymysql
import json

def query_execute(query, config_path):
    with open(config_path, 'r') as config:
        # 설정파일에서 user, passwd를 가져와 사용
        dbconf = json.load(fp=config)
        #암호화되어 있는 블록암호화 키를 복호화 해서 가져오는
        #사용자 정의 함수
        blockKey = get_decrypt_key('blockKey')
        # 설정파일에 암호화되어 있는 값을 가져와 복호화한 후에 사용
        dbUser = decrypt(blockKey, dbconf['user'])
        dbPasswd = decrypt(blockKey, dbconf['passwd'])
        
        dbconn = pymysql.connect(host=dbconf['host'], port=dbconf['port'], user=dbUser, pass
        wd=dbPasswd, db=dbconf['db_name'], charset='utf8')
        curs = dbconn.cursor()
        curs.execute(query)
        dbconn.commit()
        dbconn.close()

##### 충분하지 않은 키 길이 사용 (p.84)

In [ ]:
from Crypto.PublicKey import RSA, DSA, ECC
from tinyec import registry
import secrets

def make_rsa_key_pair():
    # RSA키 길이를 2048 비트 이상으로 길게 설정
    private_key = RSA.generate(2048)
    public_key = private_key.publickey()
    
def make_ecc():
    # ECC 키 길이를 256 비트 이상으로 설정.
    ecc_curve = registry.get_curve('secp256r1')
    private_key = secrets.randbelow(ecc_curve.field.n)
    public_key = private_key * ecc_curve.g

##### 적절하지 않은 난수 값 사용 (p.87)

In [ ]:
import secrets

def get_otp_number():
    random_str = ''
    # 보안기능에 적합한 난수 생성용 secrets 라이브러리 사용
    for i in range(6):
        random_str += str(secrets.randbelow(10))
    return random_str

In [ ]:
import secrets
import string

def generate_session_key():
    RANDOM_STRING_CHARS = string.ascii_letters+string.digits
    # 보안기능과 관련된 난수는 secrets 라이브러리를 사용해야 안전하다.
    return "".join(secrets.choice(RANDOM_STRING_CHARS) for i in range(32))

##### 취약한 비밀번호 허용 (p.89)

In [ ]:
from flask import request, redirect
from Models import User
from Models import db
import re

@app.route('/register', methods=['POST'])
def register():
    userid = request.form.get('userid')
    password = request.form.get('password')
    confirm_password = request.form.get('confirm_password')

    if password != confirm_password:
        return make_response("비밀번호가 일치하지 않습니다.", 200)

    if not check_password(password):
        return make_response("비밀번호 조합규칙에 맞지 않습니다.", 200)
    else:
        usertable=User()
        usertable.userid = userid
        usertable.password = password
        
        db.session.add(usertable)
        db.session.commit()
        return make_response("회원가입 성공", 200)

def check_password(password):
    # 2종 이상 문자로 구성된 8자리 이상 비밀번호 검사 정규식
    PT1 = re.compile('^(?=.*[A-Z])(?=.*[a-z])[A-Za-z\d!@#$%^&*]{8,}$')
    PT2 = re.compile('^(?=.*[A-Z])(?=.*\d)[A-Za-z\d!@#$%^&*]{8,}$')
    PT3 = re.compile('^(?=.*[A-Z])(?=.*[!@#$%^&*])[A-Za-z\d!@#$%^&*]{8,}$')
    PT4 = re.compile('^(?=.*[a-z])(?=.*\d)[A-Za-z\d!@#$%^&*]{8,}$')
    PT5 = re.compile('^(?=.*[a-z])(?=.*[!@#$%^&*])[A-Za-z\d!@#$%^&*]{8,}$')
    PT6 = re.compile('^(?=.*\d)(?=.*[!@#$%^&*])[A-Za-z\d!@#$%^&*]{8,}$')

    # 문자 구성 상관없이 10자리 이상 비밀번호 검사 정규식
    PT7 = re.compile('^[A-Za-z\d!@#$%^&*]{10,}$')
    
    for pattern in [PT1, PT2, PT3, PT4, PT5, PT6, PT7]:
        if pattern.match(password):
            return True
    return False

Django 프레임워크의 VALIDATORS 사용

In [ ]:
import re
from django.core.exceptions import ValidationError
from django.utils.translation import ugettext as _

class CustomValidator(object):
    def validate(self, password, user=None):
        # 2종 이상 문자로 구성된 8자리 이상 비밀번호 검사 정규식
        PT1 = re.compile('^(?=.*[A-Z])(?=.*[a-z])[A-Za-z\d!@#$%^&*]{8,}$')
        PT2 = re.compile('^(?=.*[A-Z])(?=.*\d)[A-Za-z\d$@$!%*?&]{8,}$')
        PT3 = re.compile('^(?=.*[A-Z])(?=.*[!@#$%^&*])[A-Za-z\d!@#$%^&*]{8,}$')
        PT4 = re.compile('^(?=.*[a-z])(?=.*\d)[A-Za-z\d!@#$%^&*]{8,}$')
        PT5 = re.compile('^(?=.*[a-z])(?=.*[!@#$%^&*])[A-Za-z\d!@#$%^&*]{8,}$')
        PT6 = re.compile('^(?=.*\d)(?=.*[!@#$%^&*])[A-Za-z\d!@#$%^&*]{8,}$')
        
        # 문자 구성 상관없이 10자리 이상 비밀번호 검사 정규식
        PT7 = re.compile('^[A-Za-z\d!@#$%^&*]{10,}$')
        for pattern in [PT1, PT2, PT3, PT4, PT5, PT6, PT7]:
            if pattern.match(password):
                return None
        raise ValidationError(
                _("비밀번호 조합규칙에 적합하지 않습니다.."),
                code='improper_password',
                )
    def get_help_text(self):
        return _(
        "비밀번호는 영문 대문자, 소문자, 숫자, 특수문자 조합 중 2가지 이상 8자리이거나 문자 구성 상
        관없이 10자리 이상이어야 합니다."
        )

##### 사용자 하드디스크에 저장되는 쿠키를 통한 정보 노출 (p.93)

In [ ]:
from django.http import HttpResponse

def remind_user_state(request):
    res = HttpResponse()
    #쿠키의 만료시간을 적절하게 부여하고 secure 옵션을 활성화 한다.
    res.set_cookie('rememberme', 1, max_age=60*60, secure=True, httponly=True)
    return res

In [ ]:
# settings.py

SESSION_COOKIE_AGE = 60 * 60 # default 2weeks

SESSION_COOKIE_HTTPONLY = True # default True

SEESION_COOKIE_SECURE = True # default False

##### 주석문 안에 포함된 시스템 주요정보 (p.95)

In [ ]:
def user_login(id, passwd):
    # 주석문에 포함된 민감한 정보는 삭제
    result = login(id, passwd)
    return result

##### 솔트 없이 일방향 해쉬 함수 사용 (p.97)

In [ ]:
import hashlib
import secrets

def get_hash_from_pwd(pw):
    # 비밀번호와 같이 길이가 짧은 값을 이용하여
    # 고강도의 해시를 생성하기 위해서는 salt값을
    # 사용하면 강도높은 해시를 생성할 수 있다
    salt = secrets.token_hex(32)
    h = hashlib.sha256(salt.encode() + pw.encode())
    
    return h.digest(), salt

##### 무결성 검사없는 코드 다운로드 (p.100)

In [ ]:
import requests
import hashlib

def execute_remote_code():
    url = "https://www.somewhere.com/storage/code.py"
    remote_code_hash = config.get('HASH', 'file_hash')

    #원격 코드 다운로드
    file = requests.get(url)
    remote_code = file.content

    sha = hashlib.sha256()
    sha.update(remote_code)

    #다운로드 받은 파일의 해시값 검증
    if sha.hexdigest() != remote_code_hash:
        raise Exception(‘파일이 손상되었습니다.’)
    
    file_name = 'save.py'
    with open(file_name, 'wb') as f
        f.write(file.content)
    ......

##### 반복된 인증시도 제한 기능 부재 (p.102)

In [ ]:
import hashlib
from django.shortcuts import render

LOGIN_TRY_LIMIT = 5

def login(request):
    user_id = request.POST.get('user_id', '')
    user_pw = request.POST.get('user_pw', '')
    
    # 로그인 실패기록 가져오기
    login_fail = models.LoginFail.objects.filter(user_id)
    # 로그인 실패횟수 초과로 인해 잠금된 계정에 대한 인증 시도 제한
    if login_fail.count() >= LOGIN_TRY_LIMIT:
        return render(request, '/account_lock.html', {'state':'account_lock'})
    else:
        sha = hashlib.sha256()
        sha.update(user_pw)